# Sentiment Analysis dari Instagram comments calon Presiden 2024 sebelum dan sesudah deklrasi calon wakil presiden menggunakan Naive Bayes


## Tujuan : mengetahui perbedaan sentiment dari komentar instagram sebelum dan sesudah deklarasi calon wakil presiden


<ul>
    <li>
    Aurelius Ivan Wijaya (00000054769)
    </li>
    <li>
    Rajendra Abhinaya (00000060445)
    </li>
    <li>
    Maecyntha Irelynn Tantra (00000055038)
    </li>
    <li>
    Patricia theodora (00000054093)
    </li>
<ul>

# Data Collection

apa sih yang sebenernya kita cari?
* web scrapping algoritm untuk data primer (ivan)
* labeling (pat)
* data sekunder (mae)
* stopword library indonesia (abhi)
* cari jurnal referensi yang sudah, sebagai literature review (all, min 4 per person)

In [61]:
import pandas as pd
import numpy as np

# import dataset    
# [ Primary Dataset ]
anies_before = pd.DataFrame(pd.read_csv('./Dataset/Anies/anies_before.csv')['comments'])
anies_after = pd.DataFrame(pd.read_csv('./Dataset/Anies/anies_after.csv')['comments'])
ganjar_before = pd.DataFrame(pd.read_csv('./Dataset/Ganjar/ganjar_before.csv')['comments'])
ganjar_after = pd.DataFrame(pd.read_csv('./Dataset/Ganjar/ganjar_after.csv')['comments'])
prabowo_before = pd.DataFrame(pd.read_csv('./Dataset/Prabowo/prabowo_before.csv')['comments'])
prabowo_after = pd.DataFrame(pd.read_csv('./Dataset/Prabowo/prabowo_after.csv')['comments'])

# [ Secondary Dataset ]
instagram_cyber_comments = pd.read_csv('./Dataset/dataset_komentar_instagram_cyberbullying.csv')
tweet_tv = pd.read_csv('./Dataset/dataset_tweet_sentimen_tayangan_tv.csv')
tweet_pilkada = pd.read_csv('./Dataset/dataset_tweet_sentiment_pilkada_DKI_2017.csv')
tweet_opini_film = pd.read_csv('./Dataset/dataset_tweet_sentiment_opini_film.csv')
tweet_cellular = pd.read_csv('./Dataset/dataset_tweet_sentiment_cellular_service_provider.csv')
prastyo_sentiment = pd.read_csv('./Dataset/prastyo-sentiment_all.csv')
# prastyo_sentiment_covid = pd.read_csv('./Dataset/prastyo-sentiment_posneg.csv')

In [62]:
display(prastyo_sentiment.head())

,comments,label
0,Yuppp Karena UTANG Pemerintah jg UTANG Bangsa ...,neg
1,Yuks.. kawal kebijakan pemerintah jangan sampa...,pos
2,Yukk sama-sama bahu membahu membantuu pemerint...,pos
3,Yuk sahabat kita samasama menjaga jarak antar ...,pos
4,Yuk sahabat kita sama sama menjaga jarak antar...,pos


## Data Integration

In [63]:
# 1: positive, 0: negative 

# data integration
# instagram_cyber_comments
# instagram_cyber_comments['label'] = 1
# change label name to 'comments'
instagram_cyber_comments.rename(columns={'Instagram Comment Text': 'comments'}, inplace=True)
# mapping sentiment
instagram_cyber_comments['Sentiment'] = instagram_cyber_comments['Sentiment'].map({'positive': 1, 'negative': 0})
instagram_cyber_comments['label'] = instagram_cyber_comments['Sentiment'].astype(int)
# drop unused columns
instagram_cyber_comments.drop(columns=['Id', 'Sentiment'], inplace=True)
# display(instagram_cyber_comments.head())
#change data type to string to ensure all data type is string
instagram_cyber_comments['comments'] = instagram_cyber_comments['comments'].astype(str)

# tweet_tv
# change label name to 'comments'
tweet_tv.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_tv['Sentiment'] = tweet_tv['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_tv['label'] = tweet_tv['Sentiment'].astype(int)
tweet_tv.rename(columns={'Text Tweet': 'comments'}, inplace=True)
tweet_tv.drop(columns=['Id', 'Sentiment', "Jumlah Retweet", "Acara TV"], inplace=True)
#change data type to string to ensure all data type is string
tweet_tv['comments'] = tweet_tv['comments'].astype(str)

# tweet_pilkada
# tweet_pilkada add new column 'label'
# tweet_pilkada['label'] = tweet_pilkada['Sentiment'].map({'positive': 1, 'negative': 0}).astype(int)
tweet_pilkada['Sentiment'] = tweet_pilkada['Sentiment'].map({'positive': 1, 'negative': 0}).astype(int)
# tweet_pilkada['label'] = tweet_pilkada['Sentiment'].astype(int)
tweet_pilkada = tweet_pilkada[['Sentiment', 'Text Tweet']]
tweet_pilkada.rename(columns={'Text Tweet': 'comments'}, inplace=True)
tweet_pilkada.rename(columns={'Sentiment': 'label'}, inplace=True)
#change data type to string to ensure all data type is string
tweet_pilkada['comments'] = tweet_pilkada['comments'].astype(str)

# tweet_opini_film
# change label name to 'comments'
tweet_opini_film.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_opini_film['Sentiment'] = tweet_opini_film['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_opini_film['label'] = tweet_opini_film['Sentiment'].astype(int)
tweet_opini_film.drop(columns=['Id', 'Sentiment'], inplace=True)
tweet_opini_film.rename(columns={'Text Tweet': 'comments'}, inplace=True)
#change data type to string to ensure all data type is string
tweet_opini_film['comments'] = tweet_opini_film['comments'].astype(str)

# tweet_cellular
# change label name to 'comments'
tweet_cellular.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_cellular['Sentiment'] = tweet_cellular['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_cellular['label'] = tweet_cellular['Sentiment'].astype(int)
tweet_cellular.drop(columns=['Id', 'Sentiment'], inplace=True)
tweet_cellular.rename(columns={'Text Tweet': 'comments'}, inplace=True)

# prastyo_sentiment
map = {'pos': 1, 'neg': 0, 'neu': 1}
prastyo_sentiment['label'] = prastyo_sentiment['label'].map(map)

# integrate all secondary dataset
secondary_dataset = pd.concat([
    instagram_cyber_comments,
    # tweet_tv, 
    # tweet_pilkada, 
    # tweet_opini_film, 
    # # tweet_cellular
    # prastyo_sentiment
    ], ignore_index=True)
#change data type to string to ensure all data type is string
secondary_dataset['comments'] = secondary_dataset['comments'].astype(str)

In [64]:
display(secondary_dataset.tail())

,comments,label
395,Bangga sama suami yg selalu ingat istri disela...,1
396,Apaoun pekerjaannya yg penting halal u tuk men...,1
397,Gojek itu mayoritas pegangguran yang lama gak ...,1
398,<USERNAME> aslinya cantik dan ayu loh mbak kr...,1
399,"<USERNAME> suami saya seumuran sama saya mba,...",1


# Pre-Proccessing

## Data Cleaning

### Handle Missing Value

In [65]:
def handleMissingValue(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df[df['comments'] != '']
    # remove 'NaN' value
    if('NaN' in df['comments']):
        df = df[df['comments'] != 'NaN']
    df = df[df['comments'].notna()]
    df = df.reset_index(drop=True)
    df = df[['comments']] 
    return df
# anies_before['comments'] = handleMissingValue(anies_before)
# anies_after['comments'] = handleMissingValue(anies_after)
# ganjar_before['comments'] = handleMissingValue(ganjar_before)
# ganjar_after['comments'] = handleMissingValue(ganjar_after)
# prabowo_before['comments'] = handleMissingValue(prabowo_before)
# prabowo_after['comments'] = handleMissingValue(prabowo_after)
secondary_dataset['comments'] = handleMissingValue(secondary_dataset)

In [66]:
display(secondary_dataset.head())

,comments,label
0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,0
1,Geblek lo tata...cowo bgt dibela2in balikan......,0
2,Kmrn termewek2 skr lengket lg duhhh kok labil ...,0
3,"Intinya kalau kesel dengan ATT nya, gausah ke ...",0
4,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",0


In [67]:
print(secondary_dataset.isnull().sum())
# delete duplicate data and missing value and null value
secondary_dataset = secondary_dataset.dropna()
secondary_dataset = secondary_dataset.drop_duplicates()
secondary_dataset = secondary_dataset[secondary_dataset['comments'] != '']
# remove 'NaN' value
if('NaN' in secondary_dataset['comments']):
    secondary_dataset = secondary_dataset[secondary_dataset['comments'] != 'NaN']
secondary_dataset = secondary_dataset[secondary_dataset['comments'].notna()]
secondary_dataset = secondary_dataset.reset_index(drop=True)
# secondary_dataset = secondary_dataset[['comments']]
print(secondary_dataset.isnull().sum())

display(secondary_dataset.tail())
# secondary_dataset.info()

comments    3
label       0
dtype: int64
comments    0
label       0
dtype: int64


,comments,label
392,Bangga sama suami yg selalu ingat istri disela...,1
393,Apaoun pekerjaannya yg penting halal u tuk men...,1
394,Gojek itu mayoritas pegangguran yang lama gak ...,1
395,<USERNAME> aslinya cantik dan ayu loh mbak kr...,1
396,"<USERNAME> suami saya seumuran sama saya mba,...",1


### Case Folding

handle case folding to make sure all the words are in the same case (lowercase)

In [68]:
import string
# Handle case folding
# def case_folding(data):
#     datatemps = []
#     for i in range(0, len(data)):
#         try:
#             print(data['comments'][i])
#             commentTemp = data['comments'][i]
#             datatemps.append(commentTemp.str.lower())
#             #lower case
#             commentTemp = data
#         except KeyError as e:
#             # print(f"KeyError at index {i}: {e}")
#             pass # skip the row if there is no comment
#     datatemps = {'comments': datatemps}
#     return pd.DataFrame(datatemps)

def case_folding(data):
    datatemps = []
    for i in range(0, len(data)):
        try:
            commentTemp = data['comments'][i]
            if isinstance(commentTemp, str):  # Check if the value is a string
                datatemps.append(commentTemp.lower())
        except KeyError as e:
            # print(f"KeyError at index {i}: {e}")
            pass  # skip the row if there is no comment
    datatemps = {'comments': datatemps}
    return pd.DataFrame(datatemps)
    

# anies_before['comments'] =  pd.DataFrame(case_folding(anies_before))
# anies_after['comments'] =  pd.DataFrame(case_folding(anies_after))
# ganjar_before['comments'] =  pd.DataFrame(case_folding(ganjar_before))
# ganjar_after['comments'] =  pd.DataFrame(case_folding(ganjar_after))
# prabowo_before['comments'] =  pd.DataFrame(case_folding(prabowo_before))
# prabowo_after['comments'] =  pd.DataFrame(case_folding(prabowo_after))
secondary_dataset['comments'] =  pd.DataFrame(case_folding(secondary_dataset))

display(secondary_dataset.head())

,comments,label
0,<username> tolol!! gak ada hubungan nya kegug...,0
1,geblek lo tata...cowo bgt dibela2in balikan......,0
2,kmrn termewek2 skr lengket lg duhhh kok labil ...,0
3,"intinya kalau kesel dengan att nya, gausah ke ...",0
4,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",0


### Punctuation & Number & Whitespace Removal

In [69]:
import string
import re
import pandas as pd

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
custom_dict = pd.read_csv('./Dataset/TextNormalization/customdict.csv')

stopword_factory = StopWordRemoverFactory()
stopwords = stopword_factory.get_stop_words()

stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

def clean_text(comment):
    comment = str(comment)
    
    # Remove @user
    comment = re.sub("@[^\s]+", " ", comment)
    
    # Remove punctuation
    comment = re.sub(f"[{string.punctuation}]", " ", comment)
    
    # Remove extra whitespaces
    comment = re.sub(r'\s+', ' ', comment)
    
    # Remove single characters
    comment = re.sub(r'\s+[a-zA-Z]\s+', ' ', comment)
    
    # Remove numbers
    comment = re.sub(r'\d+', '', comment)
    
    # Remove tags
    comment = re.sub("&lt;/?.*?&gt;", " &lt;&gt; ", comment)
    
    # Remove special characters and digits
    comment = re.sub("(\\d|\\W)+", " ", comment)
    
    # If a letter is repeated more than 2 times, replace it with 2 times
    comment = re.sub(r'(.)\1+', r'\1\1', comment)
    
    # Remove custom stopword custom dictionary
    for u in custom_dict['word']:
        comment = comment.replace(u, '')
    
    # Remove stopwords
    comment = ' '.join([word for word in comment.split() if word.lower() not in stopwords])

    # Lemmatization or stemming
    comment = ' '.join([stemmer.stem(word) for word in comment.split()])
    
    return comment.strip()


def remove_punct(data):
    data['comments'] = data['comments'].apply(clean_text)
    return data

# Call the function
cleaned_data = remove_punct(secondary_dataset)

# Save to CSV
cleaned_data.to_csv('./cleaned_data.csv', index=False)

secondary_dataset['comments'] = cleaned_data['comments']

In [70]:
display(cleaned_data)

,comments,label
0,tolol hubunn gugur pake hijab syar yg lo bilan...,0
1,geblek lo tata cowo bgt bain balik deww ntar d...,0
2,kmrn mewek skr lengket lg duhh kok labil bgt s...,0
3,inti kalau kesel denn att usah anak ju kasi bu...,0
4,deww permpuan lg sakit jiwa knp rus yg jd pera...,0
...,...,...
392,bang sama suami yg selalu int istri sela waktu...,1
393,apaoun kerja yg penting lal tuk nafkah keluar ...,1
394,mayoritas pengguran lama kerja trus mpir putus...,1
395,asli cantik ayu loh mbak krn aku temu langsung...,1


### Text Normalization / Noise Removal

* this one need research (ivan)
* slang word dataset that i used : https://github.com/nasalsabila/kamus-alay

* Contoh sebelum: "Para mahasiswa yang memperoleh nilai yang rendah dalam ujian tidak diizinkan untuk mengikuti ujian ulang."
* Contoh sesudah: "Mahasiswa yang memperoleh nilai rendah dalam ujian tidak diizinkan mengikuti ujian ulang."

In [71]:
import pandas as pd
indo_slang_word = pd.read_csv('./Dataset/TextNormalization/colloquial-indonesian-lexicon.csv')
indo_slang_word.head()

def replace_slang_word(doc,slang_word):
    for index in  range(0,len(doc)-1):
        index_slang = slang_word.slang==doc[index]
        formal = list(set(slang_word[index_slang].formal))
        if len(formal)==1:
            doc[index]=formal[0]
    return doc

# def text_normalization(data):
#     datatemps = []
#     for i in range(0, len(data)):
#         comment = data.iloc[i, 0]  # Access the 'comments' column in the DataFrame
#         comment = comment.split()
#         comment = replace_slang_word(comment,indo_slang_word)
#         comment = ' '.join(comment)
#         datatemps.append(comment)
#     return datatemps

def text_normalization(data):
    datatemps = []
    for i in range(0, len(data)):
        comment = str(data['comments'][i])  # Convert to string
        comment = comment.split()
        comment = replace_slang_word(comment, indo_slang_word)
        comment = ' '.join(comment)
        datatemps.append(comment)
    return datatemps

# Call the functions successively
# anies_before['comments'] = pd.DataFrame(text_normalization(anies_before))
# anies_after['comments'] = pd.DataFrame(text_normalization(anies_after))
# ganjar_before['comments'] = pd.DataFrame(text_normalization(ganjar_before))
# ganjar_after['comments'] = pd.DataFrame(text_normalization(ganjar_after))
# prabowo_before['comments'] = pd.DataFrame(text_normalization(prabowo_before))
# prabowo_after['comments'] = pd.DataFrame(text_normalization(prabowo_after))
secondary_dataset['comments'] = pd.DataFrame(text_normalization(secondary_dataset))

In [72]:
display(secondary_dataset.head())

,comments,label
0,tolol hubunn gugur pakai hijab syar yang lo bi...,0
1,geblek lo tata cowok banget bain balik deww en...,0
2,kemarin mewek sekarang lengket lagi duh kok la...,0
3,inti kalau kesel denn att usah anak ju kasih b...,0
4,deww permpuan lagi sakit jiwa kenapa rus yang ...,0


### Stopwords Removal

* this one need research (abhi)

In [73]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# download nltk
nltk.download('punkt')

def remove_stopwords(data):
    datatemps = []
    for i in range(0, len(data)):
        comment = data['comments']  # Access the 'comments' column in the DataFrame
        # print(comment)
        stop_words = set(stopwords.words('indonesian'))
        word_tokens = word_tokenize(comment[i])
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        comment = ' '.join(filtered_sentence)
        datatemps.append(comment)
    return datatemps

# Call the functions successively
# anies_before['comments'] = pd.DataFrame(remove_stopwords(anies_before))
# anies_after['comments'] = pd.DataFrame(remove_stopwords(anies_after))
# ganjar_before['comments'] = pd.DataFrame(remove_stopwords(ganjar_before))
# ganjar_after['comments'] = pd.DataFrame(remove_stopwords(ganjar_after))
# prabowo_before['comments'] = pd.DataFrame(remove_stopwords(prabowo_before))
# prabowo_after['comments'] = pd.DataFrame(remove_stopwords(prabowo_after))
secondary_dataset['comments'] = pd.DataFrame(remove_stopwords(secondary_dataset))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aurel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [74]:
display(secondary_dataset.head())

,comments,label
0,tolol hubunn gugur pakai hijab syar lo bilang ...,0
1,geblek lo tata cowok banget bain deww entar di...,0
2,kemarin mewek lengket duh labil banget sih mba...,0
3,inti kesel denn att anak ju kasih perkembann p...,0
4,deww permpuan sakit jiwa rus peran utama film ...,0


### Stemming / Lemmatization

* this one need research (mae)


In [75]:
# # # import Sastrawi package
# # from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# # # create stemmer
# # factory = StemmerFactory()
# # stemmer = factory.create_stemmer()

# # def stemming(data):
# #     datatemps = []
# #     for i in range(0, len(data)):
# #         comment = data['comments']  # Access the 'comments' column in the DataFrame
# #         # print(comment)
# #         comment = stemmer.stem(comment[i])
# #         datatemps.append(comment)
# #     return datatemps

# # # Call the functions successively
# # secondary_dataset['comments'] = pd.DataFrame(stemming(secondary_dataset))
# # display(anies_after.head())


# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# # Create stemmer
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()

# # Define the stemming function
# def stem_text(text):
#     if isinstance(text, str):
#         return stemmer.stem(text)
#     else:
#         return text

# # Convert 'comments' column to string and apply the stemming function
# secondary_dataset['comments'] = secondary_dataset['comments'].astype(str).apply(stem_text)

# # Display the updated DataFrame
# display(secondary_dataset.head())


In [76]:
# display(secondary_dataset.head())
display(secondary_dataset.head())

,comments,label
0,tolol hubunn gugur pakai hijab syar lo bilang ...,0
1,geblek lo tata cowok banget bain deww entar di...,0
2,kemarin mewek lengket duh labil banget sih mba...,0
3,inti kesel denn att anak ju kasih perkembann p...,0
4,deww permpuan sakit jiwa rus peran utama film ...,0


## Tokenization

* this one need research (mae)

In [77]:
# def tokenization(data):
#     datatemps = []
#     for i in range(0, len(data)):
#         comment = data['comments']  # Access the 'comments' column in the DataFrame
#         # print(comment)
#         comment = word_tokenize(comment[i])
#         datatemps.append(comment)
#     return datatemps
def tokenization(data):
    datatemps = []
    for i in range(0, len(data)):
        comment = data['comments'][i]  # Access the 'comments' column in the DataFrame
        comment = word_tokenize(comment)
        datatemps.append(comment)
    return datatemps

# # Call the functions successively
# # anies_before['comments'] = pd.DataFrame(tokenization(anies_before))
# # anies_after['comments'] = pd.DataFrame(tokenization(anies_after))
# # ganjar_before['comments'] = pd.DataFrame(tokenization(ganjar_before))
# # ganjar_after['comments'] = pd.DataFrame(tokenization(ganjar_after))
# # prabowo_before['comments'] = pd.DataFrame(tokenization(prabowo_before))
# # prabowo_after['comments'] = pd.DataFrame(tokenization(prabowo_after))
# # secondary_dataset['comments'] = pd.DataFrame(tokenization(secondary_dataset))   

# anies_before['comments'] = tokenization(anies_before)
# anies_after['comments'] = tokenization(anies_after)
# ganjar_before['comments'] = tokenization(ganjar_before)
# ganjar_after['comments'] = tokenization(ganjar_after)
# prabowo_before['comments'] = tokenization(prabowo_before)
# prabowo_after['comments'] = tokenization(prabowo_after)
# secondary_dataset['comments'] = tokenization(secondary_dataset)

from sklearn.feature_extraction.text import CountVectorizer

# # CountVectorizer
vectorizer = CountVectorizer()
secondary_dataset_vectorizer = tokenization(secondary_dataset)
# print(vectorizer.get_feature_names())
# # display(X_train.shape)
# print(secondary_dataset['comments'])
# display(secondary_dataset)
display(secondary_dataset_vectorizer)
secondary_dataset['comments'] = secondary_dataset_vectorizer

[['tolol',
  'hubunn',
  'gugur',
  'pakai',
  'hijab',
  'syar',
  'lo',
  'bilang',
  'bayi',
  'panas',
  'dalem',
  'hubunn',
  'woyy',
  'otak',
  'jempol',
  'lo',
  'singkron',
  'sih',
  'menulis',
  'komentar'],
 ['geblek',
  'lo',
  'tata',
  'cowok',
  'banget',
  'bain',
  'deww',
  'entar',
  'ditingl',
  'lahin',
  'tuh',
  'cowok',
  'padal',
  'oon'],
 ['kemarin',
  'mewek',
  'lengket',
  'duh',
  'labil',
  'banget',
  'sih',
  'mbak',
  'kaya',
  'abege',
  'kemarin',
  'cari',
  'sensasi',
  'biar',
  'top',
  'markotopp',
  'ertong',
  'kualitas'],
 ['inti',
  'kesel',
  'denn',
  'att',
  'anak',
  'ju',
  'kasih',
  'perkembann',
  'psikis',
  'anak',
  'orang',
  'tolol',
  'anak',
  'anak',
  'dikatai',
  'orang',
  'benci',
  'asa',
  'benci',
  'orang',
  'rus',
  'tau',
  'bates',
  'nama',
  'manusia',
  'suka',
  'ters'],
 ['deww',
  'permpuan',
  'sakit',
  'jiwa',
  'rus',
  'peran',
  'utama',
  'film',
  'ntu',
  'jeruk',
  'purut',
  'kaya',
  'kbisan

In [78]:
display(secondary_dataset.head())


,comments,label
0,"[tolol, hubunn, gugur, pakai, hijab, syar, lo,...",0
1,"[geblek, lo, tata, cowok, banget, bain, deww, ...",0
2,"[kemarin, mewek, lengket, duh, labil, banget, ...",0
3,"[inti, kesel, denn, att, anak, ju, kasih, perk...",0
4,"[deww, permpuan, sakit, jiwa, rus, peran, utam...",0


## Synthetic Minority Oversampling Technique (SMOTE)

* this one need research (ivan)
* Smote adalah sebuah tehnik yang digunakan terhadap data yang tidak seimbang

In [79]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE()
# X_train_smote, y_train_smote = smote.fit_resample(X_train_tweets_tfidf, y_train.values)
# print(X_train_smote.shape, y_train_smote.shape)

# # SMOTE on full training data
# smote = SMOTE()
# X_smote, y_smote = smote.fit_resample(X_tweets_tfidf, y.values)
# print(X_smote.shape, y_smote.shape)

# # Class Imbalance Check
# plt.pie(pd.value_counts(y_train_smote), 
#         labels=['Label 0 (Positive)', 'Label 1 (Negative)'], 
#         autopct='%0.1f%%')
# plt.axis('equal')
# plt.show()


display(secondary_dataset.head())

,comments,label
0,"[tolol, hubunn, gugur, pakai, hijab, syar, lo,...",0
1,"[geblek, lo, tata, cowok, banget, bain, deww, ...",0
2,"[kemarin, mewek, lengket, duh, labil, banget, ...",0
3,"[inti, kesel, denn, att, anak, ju, kasih, perk...",0
4,"[deww, permpuan, sakit, jiwa, rus, peran, utam...",0


## K-Fold Cross Validation

* this one need research (abhi)

In [80]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, LSTM, Dense
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Assuming 'comments' is the text data in your dataset
# X = secondary_dataset['comments']
# y = secondary_dataset['label']

# # Encode the labels
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y)

# # Tokenize the text data
# max_words = 5000  # Maximum number of words to keep
# tokenizer = Tokenizer(num_words=max_words)
# tokenizer.fit_on_texts(X)
# X_sequences = tokenizer.texts_to_sequences(X)

# # Pad sequences to a fixed length (you can adjust maxlen as needed)
# maxlen = 100  # Example: Maximum length of a sequence
# X_padded = pad_sequences(X_sequences, maxlen=maxlen)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# # Build the LSTM model
# model = Sequential()
# model.add(Embedding(input_dim=max_words, output_dim=128, input_length=maxlen))
# model.add(LSTM(128))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model
# model.fit(X_train, y_train, epochs=12, batch_size=32, validation_split=0.2)

# # Evaluate the model on the test set
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f"\nTest Accuracy: {accuracy}")

# # Predictions
# predictions = model.predict(X_test)

In [81]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
# svc 
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer

# Assuming 'comments' is the text data in your dataset
X = secondary_dataset['comments']
y = secondary_dataset['label']

# Join the list of strings into a single string for each document
X = [' '.join(comment) for comment in X]

# Use CountVectorizer to convert text data to a format suitable for RandomForestClassifier
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X).toarray()  # Convert to array

# Define 10-fold cross-validation test harness
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Instantiate your classifier
classifier = MultinomialNB(alpha=0.1)

# Use cross_val_score to perform K-Fold Cross Validation
cv_results = cross_val_score(classifier, X, y, cv=kfold)

# Print the results for each fold
best = 0

for i, accuracy in enumerate(cv_results):
    if accuracy > best:
        best = i
        print(f"Fold {i + 1}: Accuracy = {accuracy} <== BEST RESULT")
    else:
        pass
    print(f"Fold {i + 1}: Accuracy = {accuracy}")
# Print the mean and standard deviation of the cross-validation results
print(f"\nMean Accuracy: {cv_results.mean()}")
print(f"Standard Deviation: {cv_results.std()}")

# fold 6
classifier.fit(X, y)

Fold 1: Accuracy = 0.925 <== BEST RESULT
Fold 1: Accuracy = 0.925
Fold 2: Accuracy = 1.0 <== BEST RESULT
Fold 2: Accuracy = 1.0
Fold 3: Accuracy = 0.95
Fold 4: Accuracy = 0.85
Fold 5: Accuracy = 0.925
Fold 6: Accuracy = 0.825
Fold 7: Accuracy = 0.85
Fold 8: Accuracy = 0.9230769230769231
Fold 9: Accuracy = 1.0
Fold 10: Accuracy = 0.8205128205128205

Mean Accuracy: 0.9068589743589743
Standard Deviation: 0.06383987843734874


MultinomialNB(alpha=0.1)

In [82]:
from sklearn.model_selection import GridSearchCV

# Hyperparameter tuning for RandomForestClassifier
model_param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]}
model_grid_search = GridSearchCV(MultinomialNB(), model_param_grid, cv=kfold)
model_grid_search.fit(X, y)

# Print the best parameters and corresponding accuracy
print("\nBest parameters for RandomForestClassifier:", model_grid_search.best_params_)
print("Best accuracy for RandomForestClassifier:", model_grid_search.best_score_)



Best parameters for RandomForestClassifier: {'alpha': 0.1}
Best accuracy for RandomForestClassifier: 0.9068589743589743


In [83]:
# # for LSTM

# # Define the prediction function
# def prediction(data):
#     # Tokenize the input data
#     tokenized = tokenizer.texts_to_sequences(data)
    
#     # Pad sequences to the same length as in training
#     padded = pad_sequences(tokenized, maxlen=maxlen)
    
#     # Make predictions using the trained model
#     predictions = model.predict(padded)

#     # Convert predictions to labels
#     labels = (predictions > 0.5).astype(int)
    
#     return label_encoder.inverse_transform(labels)

# # Call the function successively
# text = ['anak bego']
# pred = prediction(text)

# print(pred)

In [84]:
# text = ['anak bego']
# pred = prediction(text)
# print(pred)

In [90]:
def prediction(data):
    tokenized = []
    for i in range(0, len(data)):
        tokenized.extend(word_tokenize(data[i]))

    # Transform using the same vectorizer used during training
    text = vectorizer.transform([" ".join(tokenized)]).toarray()

    return classifier.predict(text)

# Call the function successively
text = ['jangan gitu dong']
pred = prediction(text)

print(pred)

[1]


In [86]:
# display(ganjar_after.head())
# # ganjar_after['comments'] = tokenization(ganjar_after)
# negative = 0
# positive = 0
# for i in range(0, len(ganjar_after)):
#     pred = prediction(ganjar_after['comments'][i])
#     print(pred, ganjar_after['comments'][i])
#     if pred == 0:
#         negative += 1
#     else:
#         positive += 1
# print(negative, positive)


## Workload Distribution?

# Modeling

<!-- * kemungkinan Binomial Naive Bayes -->

In [87]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import GaussianNB

# # Assuming X_train is a list of strings (or can be converted to strings)
# text_data = [str(item) for item in X_train]

# # Create a TfidfVectorizer
# vectorizer = TfidfVectorizer(min_df=2)  # Adjust min_df as needed

# # Vectorize the training set
# X_train_vectorized = vectorizer.fit_transform(text_data)

# # Assuming X_test is a list of strings (or can be converted to strings)
# X_test = [str(item) for item in X_test]

# # Train the Gaussian Naive Bayes model
# gnb = GaussianNB()
# gnb.fit(X_train_vectorized.toarray(), y_train)  # Convert sparse matrix to dense array

# # Vectorize the test set
# X_test_vectorized = vectorizer.transform(X_test)

# # Make predictions on the test set
# predictions = gnb.predict(X_test_vectorized.toarray())  # Convert sparse matrix to dense array

# # Continue with evaluation as needed


NameError: name 'X_train' is not defined

In [ ]:
# # accuracy
# from sklearn.metrics import accuracy_score
# X_test_vectorized = vectorizer.transform(X_test)
# y_pred = gnb.predict(X_test_vectorized.toarray())
# print('Accuracy: ', accuracy_score(y_test, y_pred))

Accuracy:  0.6622222222222223


In [ ]:
# # Multinomial Naive Bayes
# from sklearn.naive_bayes import MultinomialNB

# # Train the model
# mnb = MultinomialNB(alpha=0.8, class_prior=None, fit_prior=True)
# mnb.fit(X_train_vectorized, y_train)

MultinomialNB(alpha=0.8)

In [ ]:
# # accuracy
# from sklearn.metrics import accuracy_score
# y_pred = mnb.predict(X_test_vectorized.toarray())
# print('Accuracy: ', accuracy_score(y_test, y_pred))

# # 0.7681818181818182

Accuracy:  0.7466666666666667


In [ ]:
# # Bernoulli Naive Bayes
# from sklearn.naive_bayes import BernoulliNB

# # Train the model
# bnb = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
# bnb.fit(X_train_vectorized, y_train)

BernoulliNB(alpha=1)

In [ ]:
# # accuracy
# from sklearn.metrics import accuracy_score
# y_pred = bnb.predict(X_test_vectorized.toarray())
# print('Accuracy: ', accuracy_score(y_test, y_pred))

Accuracy:  0.7155555555555555


### Random Forest Classifier

In [ ]:
# # Random Forest Classifier

# from sklearn.ensemble import RandomForestClassifier

# # Train the model
# rfc = RandomForestClassifier(n_estimators=100, random_state=42)
# rfc.fit(X_train_vectorized, y_train)

# # accuracy
# from sklearn.metrics import accuracy_score
# y_pred = rfc.predict(X_test_vectorized.toarray())

# print('Accuracy: ', accuracy_score(y_test, y_pred))

Accuracy:  0.6933333333333334


In [ ]:
# # SVM Classifier
# from sklearn.svm import SVC

# # Train the model
# svm = SVC()

# svm.fit(X_train_vectorized, y_train)

# # accuracy

# from sklearn.metrics import accuracy_score
# y_pred = svm.predict(X_test_vectorized.toarray())
# print('Accuracy: ', accuracy_score(y_test, y_pred))

Accuracy:  0.7066666666666667


# Model Evaluation

* this one need research (pat)

# Analysis the data from primary data

In [ ]:
# def predict_sentiment(model, vectorizer, text):
#     # Vectorize the input text
#     text_vectorized = vectorizer.transform([text])
#     # Make predictions
#     prediction = model.predict(text_vectorized.toarray())
    
#     return prediction[0]

# # Test the model
# # print(predict_sentiment(gnb, vectorizer, text))
# # anies before

# def predict_batch(data):
#     positive_count = 0
#     negative_count = 0
#     for i in range(0, len(data)):
#         prediction = predict_sentiment(gnb, vectorizer, data['comments'][i])
#         if prediction == 1:
#             positive_count += 1
#         else:
#             negative_count += 1
#         # print('sentiment', prediction, data['comments'][i])
#     print('komentar positive: ', positive_count)
#     print('komentar negative: ', negative_count)


# predict_batch(ganjar_before)

ValueError: np.nan is an invalid document, expected byte or unicode string.

## Modal Comprarison

# Conclusion